# **Notebook From:**
# **https://github.com/WangRongsheng**
# **https://github.com/Vision-CAIR/MiniGPT-4**


## Now you can use MiniGPT-4 13B/7B in Google Colab

## <font color="#dd0000">Notice</font>

1. You should be a **Google Colab Pro** user.
2. You must use **high level GPU**.

Otherwise you will not be able to run MiniGPT-4 in colab!!!


| Model Variant | Created From |
|---|---|
| 7B: `wangrongsheng/MiniGPT-4-LLaMA-7B`| `vicuna-7b-delta-v0` and `llama-7b-hf` |
| 13B:  `wangrongsheng/MiniGPT-4-LLaMA` |`vicuna-13b-delta-v0` and `llama-13b-hf` |

In [ ]:
#@markdown # Check GPU
!nvidia-smi

In [ ]:
# source https://github.com/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb
#@markdown # Connect Google Drive
from google.colab import drive
import os

Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("Connecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive 
else:
  mainpth="MyDrive"

print("\u2714 Google Drive connected successfully")


In [ ]:
#@markdown # Clone repo

import os
import time

mainpth = "MyDrive"

if not os.path.exists("/content/gdrive"):
    print('Gdrive not connected, using colab storage ...')
    time.sleep(4)
    !mkdir -p /content/gdrive/$mainpth
    Shared_Drive = ""

if Shared_Drive != "" and not os.path.exists("/content/gdrive/Shareddrives"):
    print('Shared drive not detected, using default MyDrive')
    mainpth = "MyDrive"

%cd /content/gdrive/$mainpth
!git clone https://github.com/Vision-CAIR/MiniGPT-4.git

print("Done")

In [ ]:
#@markdown # Install Requirements 
%cd MiniGPT-4/
!wget -O requirements.txt https://raw.githubusercontent.com/WangRongsheng/Use-LLMs-in-Colab/main/MiniGPT-4/requirements.txt
!pip install -qr requirements.txt

!pip install -q salesforce-lavis
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q gradio==3.27.0

!pip install -q h5py
!pip install -q typing-extensions
!pip install -q wheel
!pip install -q git+https://github.com/huggingface/transformers.git -U

In [ ]:
#@markdown # Choose and download pretrained_minigpt4
Model_Version = "7B" #@param ["13B", "7B"]

if Model_Version == "13B":
    !wget -O models/pretrained_minigpt4.pth https://huggingface.co/wangrongsheng/MiniGPT4/resolve/main/pretrained_minigpt4.pth
elif Model_Version == "7B":
    !wget -O models/prerained_minigpt4_7b.pth https://huggingface.co/wangrongsheng/MiniGPT4-7B/resolve/main/prerained_minigpt4_7b.pth

#@title Choose your model
llama_model = Model_Version

if llama_model == "13B":
    !sed -i 's|llama_model:.*|llama_model: "wangrongsheng/MiniGPT-4-LLaMA"|' minigpt4/configs/models/minigpt4.yaml
    !sed -i 's|ckpt:.*|ckpt: "models/pretrained_minigpt4.pth"|' eval_configs/minigpt4_eval.yaml
else:
    !sed -i 's|llama_model:.*|llama_model: "wangrongsheng/MiniGPT-4-LLaMA-7B"|' minigpt4/configs/models/minigpt4.yaml
    !sed -i 's|ckpt:.*|ckpt: "models/prerained_minigpt4_7b.pth"|' eval_configs/minigpt4_eval.yaml


In [ ]:
#@markdown # Run MiniGPT-4
#@markdown #### If the cell stops/error during startup you probably need more VRAM 
!python demo.py --cfg-path eval_configs/minigpt4_eval.yaml --gpu-id 0